In [32]:
!uv pip install kagglehub
!UV pip install kagglehub pandas sqlalchemy
import pandas as pd
import sqlite3
import os
import kagglehub

# Download dataset using kagglehub 
print("Downloading dataset from Kaggle Hub...")
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print(f"Dataset downloaded to: {path}")

# Define full file paths using the path from kagglehub
# os.path.join combines the directory path and filename 
customers_csv = os.path.join(path, 'olist_customers_dataset.csv')
orders_csv = os.path.join(path, 'olist_orders_dataset.csv')
order_items_csv = os.path.join(path, 'olist_order_items_dataset.csv')
products_csv = os.path.join(path, 'olist_products_dataset.csv')
print("\nFull file paths defined.")

# Load Data 
try:
    customers_df = pd.read_csv(customers_csv)
    orders_df = pd.read_csv(orders_csv)
    order_items_df = pd.read_csv(order_items_csv)
    products_df = pd.read_csv(products_csv)
    print("CSV files loaded into pandas DataFrames.")
except FileNotFoundError as e:
    print(f"Error loading CSVs: {e}")
    print("Please ensure the dataset was downloaded correctly by kagglehub.")
    raise

# Create SQLite Database and load tables
db_name = 'olist.db'
if os.path.exists(db_name):
    os.remove(db_name)

conn = sqlite3.connect(db_name)
print(f"Database '{db_name}' created.")

customers_df.to_sql('customers', conn, index=False, if_exists='replace')
orders_df.to_sql('orders', conn, index=False, if_exists='replace')
order_items_df.to_sql('order_items', conn, index=False, if_exists='replace')
products_df.to_sql('products', conn, index=False, if_exists='replace')

print("\nSetup complete. Database is ready.")

Using Python 3.9.6 environment at: flexenv
Audited 1 package in 11ms
Using Python 3.9.6 environment at: flexenv
Audited 3 packages in 3ms
Dataset downloaded to: /Users/hc/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2

Full file paths defined.
CSV files loaded into pandas DataFrames.
Database 'olist.db' created.

Setup complete. Database is ready.


In [33]:
query = """
    SELECT
        order_id,
        customer_id,
        order_purchase_timestamp
    FROM
        orders
    ORDER BY
        order_purchase_timestamp DESC
    LIMIT 5;
"""

recent_orders = pd.read_sql_query(query, conn)
print("5 most recent orders:")
recent_orders

5 most recent orders:


,order_id,customer_id,order_purchase_timestamp
0,10a045cdf6a5650c21e9cfeb60384c16,a4b417188addbc05b26b72d5e44837a1,2018-10-17 17:30:18
1,b059ee4de278302d550a3035c4cdb740,856336203359aa6a61bf3826f7d84c49,2018-10-16 20:16:02
2,a2ac6dad85cf8af5b0afb510a240fe8c,4c2ec60c29d10c34bd49cb88aa85cfc4,2018-10-03 18:55:29
3,616fa7d4871b87832197b2a137a115d2,bf6181a85bbb4115736c0a8db1a53be3,2018-10-01 15:30:09
4,392ed9afd714e3c74767d0c4d3e3f477,2823ffda607a2316375088e0d00005ec,2018-09-29 09:13:03


In [34]:
query = """
    SELECT
        order_id,
        order_status
    FROM
        orders
    WHERE
        order_status = 'canceled';
"""

canceled_orders = pd.read_sql_query(query, conn)
print(f"Found {len(canceled_orders)} canceled orders.")
canceled_orders.head()

Found 625 canceled orders.


,order_id,order_status
0,1b9ecfe83cdc259250e1a8aca174f0ad,canceled
1,714fb133a6730ab81fa1d3c1b2007291,canceled
2,3a129877493c8189c59c60eb71d97c29,canceled
3,00b1cb0320190ca0daa2c88b35206009,canceled
4,ed3efbd3a87bea76c2812c66a0b32219,canceled


In [35]:
query = """
    SELECT
        order_status,
        COUNT(order_id) AS number_of_orders
    FROM
        orders
    GROUP BY
        order_status
    ORDER BY
        number_of_orders DESC;
"""

status_counts = pd.read_sql_query(query, conn)
print("Order counts by status:")
status_counts

Order counts by status:


,order_status,number_of_orders
0,delivered,96478
1,shipped,1107
2,canceled,625
3,unavailable,609
4,invoiced,314
5,processing,301
6,created,5
7,approved,2


In [36]:
query = """
    SELECT
        product_category_name,
        COUNT(product_id) AS number_of_products
    FROM
        products
    WHERE
        product_category_name IS NOT NULL -- Good practice to filter out missing data
    GROUP BY
        product_category_name
    ORDER BY
        number_of_products DESC
    LIMIT 10;
"""

category_counts = pd.read_sql_query(query, conn)
print("Top 10 product categories by number of products:")
category_counts

Top 10 product categories by number of products:


,product_category_name,number_of_products
0,cama_mesa_banho,3029
1,esporte_lazer,2867
2,moveis_decoracao,2657
3,beleza_saude,2444
4,utilidades_domesticas,2335
5,automotivo,1900
6,informatica_acessorios,1639
7,brinquedos,1411
8,relogios_presentes,1329
9,telefonia,1134


In [37]:
query = """
    SELECT
        oi.price,
        p.product_category_name
    FROM
        order_items AS oi
    JOIN
        products AS p ON oi.product_id = p.product_id
    WHERE
        p.product_category_name IS NOT NULL
    ORDER BY
        oi.price DESC
    LIMIT 10;
"""

expensive_items = pd.read_sql_query(query, conn)
print("Categories of the 10 most expensive items sold:")
expensive_items

Categories of the 10 most expensive items sold:


,price,product_category_name
0,6735.00,utilidades_domesticas
1,6729.00,pcs
2,6499.00,artes
3,4799.00,eletroportateis
4,4690.00,eletroportateis
5,4590.00,pcs
6,4399.87,instrumentos_musicais
7,4099.99,consoles_games
8,4059.00,esporte_lazer
9,3999.90,relogios_presentes


In [38]:
query = """
    SELECT
        oi.order_id,
        c.customer_city,
        c.customer_state,
        SUM(oi.price) as order_total -- We need to sum items in the same order
    FROM
        order_items AS oi
    JOIN
        orders AS o ON oi.order_id = o.order_id
    JOIN
        customers AS c ON o.customer_id = c.customer_id
    GROUP BY
        oi.order_id -- Group all items from the same order together
    ORDER BY
        order_total DESC
    LIMIT 5;
"""

top_order_cities = pd.read_sql_query(query, conn)
print("Cities for the 5 highest-value orders:")
top_order_cities

Cities for the 5 highest-value orders:


,order_id,customer_city,customer_state,order_total
0,03caa2c082116e1d31e67e9ae3700499,rio de janeiro,RJ,13440.0
1,736e1922ae60d0d6a89247b851902527,vila velha,ES,7160.0
2,0812eb902a67711a1cb742b3cdaa65ae,campo grande,MS,6735.0
3,fefacc66af859508bf1a7934eab1e97f,vitoria,ES,6729.0
4,f5136e38d1a14a4dbd87dff67da82701,marilia,SP,6499.0


In [ ]:
query = """
    SELECT
        COUNT(product_id) as num_products,
        CASE
            WHEN price < 50 THEN 'Cheap'
            WHEN price >= 50 AND price < 200 THEN 'Affordable'
            ELSE 'Expensive'
        END AS price_tier
    FROM
        order_items
    GROUP BY
        price_tier;
"""

price_tiers = pd.read_sql_query(query, conn)
print("Product counts by price tier:")
price_tiers

Product counts by price tier:


,num_products,price_tier
0,60153,Affordable
1,39024,Cheap
2,13473,Expensive


In [41]:
query = """
    SELECT
        p.product_category_name,
        SUM(oi.price) as total_revenue
    FROM
        order_items AS oi
    JOIN
        products AS p ON oi.product_id = p.product_id
    GROUP BY
        p.product_category_name
    HAVING
        total_revenue > 200000 -- Filter on the aggregated value
    ORDER BY
        total_revenue DESC;
"""

high_revenue_categories = pd.read_sql_query(query, conn)
print("Categories with > $200k in revenue:")
high_revenue_categories

Categories with > $200k in revenue:


,product_category_name,total_revenue
0,beleza_saude,1258681.34
1,relogios_presentes,1205005.68
2,cama_mesa_banho,1036988.68
3,esporte_lazer,988048.97
4,informatica_acessorios,911954.32
5,moveis_decoracao,729762.49
6,cool_stuff,635290.85
7,utilidades_domesticas,632248.66
8,automotivo,592720.11
9,ferramentas_jardim,485256.46


In [ ]:
query = """
    -- Define a CTE to first find all orders related to health & beauty
    WITH health_beauty_orders AS (
        SELECT
            oi.order_id,
            oi.price
        FROM
            order_items AS oi
        JOIN
            products AS p ON oi.product_id = p.product_id
        WHERE
            p.product_category_name = 'beleza_saude'
    )
    -- Now, query from this temporary table to get customer details
    SELECT
        c.customer_unique_id, -- A better ID for tracking a single person
        c.customer_city,
        SUM(hbo.price) AS total_spent_on_health_beauty
    FROM
        health_beauty_orders AS hbo
    JOIN
        orders AS o ON hbo.order_id = o.order_id
    JOIN
        customers AS c ON o.customer_id = c.customer_id
    GROUP BY
        c.customer_unique_id
    ORDER BY
        total_spent_on_health_beauty DESC
    LIMIT 5;
"""

top_health_customers = pd.read_sql_query(query, conn)
print("Top 5 Customers in the Health & Beauty Category:")
top_health_customers

Top 5 Customers in the Health & Beauty Category:


,customer_unique_id,customer_city,total_spent_on_health_beauty
0,7a96eb0a685f5c19b7dad29fc802aa64,santa maria,3124.00
1,d9f8d6be11fb416742c9e35e725303f7,ourinhos,2899.00
2,c38b132f1c68c14c0d3b32e66b796eba,divinopolis,2799.65
3,34ff67eb3466a779b2f6b6fd56a3b953,sao paulo,2749.65
4,8ad6fdb3d658f916f444aaf55a49bb29,brasilia,2699.00


In [ ]:

query = """
    -- CTE to join orders and customers and then find the previous order date
    WITH orders_with_previous_date AS (
        SELECT
            c.customer_unique_id, -- The CORRECT ID to track a person
            o.order_id,
            o.order_purchase_timestamp AS current_order_date,
            -- We partition by the UNIQUE id and order by date
            LAG(o.order_purchase_timestamp, 1) OVER (
                PARTITION BY c.customer_unique_id
                ORDER BY o.order_purchase_timestamp
            ) AS previous_order_date
        FROM
            orders AS o
        -- We MUST JOIN to get the correct customer identifier
        JOIN
            customers AS c ON o.customer_id = c.customer_id
    )
    -- Now, select from the CTE and calculate the difference
    SELECT
        customer_unique_id,
        order_id,
        current_order_date,
        previous_order_date,
        -- Calculate the difference in days
        ROUND(
            JULIANDAY(current_order_date) - JULIANDAY(previous_order_date),
            2
        ) AS days_since_last_order
    FROM
        orders_with_previous_date
    -- We only care about orders that HAVE a previous order
    WHERE
        previous_order_date IS NOT NULL
    -- Order by customer to see the repeat purchases together
    ORDER BY
        customer_unique_id
    LIMIT 15;
"""

time_between_orders = pd.read_sql_query(query, conn)
print("Time passed between consecutive orders for customers:")
time_between_orders

Time passed between consecutive orders for customers:


,customer_unique_id,order_id,current_order_date,previous_order_date,days_since_last_order
0,00172711b30d52eea8b313a7f2cced02,c306eca42d32507b970739b5b6a5a33a,2018-08-13 09:14:07,2018-07-28 00:23:49,16.37
1,004288347e5e88a27ded2bb23747066c,08204559bebd39e09ee52dcb56d8faa2,2018-01-14 07:36:54,2017-07-27 14:13:03,170.72
2,004b45ec5c64187465168251cd1c9c2f,9392c5e72885ad5aba87e6223ca9838d,2018-05-26 19:42:48,2017-09-01 12:11:23,267.31
3,0058f300f57d7b93c477a131a59b36c3,81a93b2fa39e104b865b2bc471c16008,2018-03-22 18:09:41,2018-02-19 17:11:34,31.04
4,00a39521eb40f7012db50455bf083460,cea3e6c11eb60acb9d8d4d51694832f8,2018-06-03 10:12:57,2018-05-23 20:14:21,10.58
5,00cc12a6d8b578b8ebd21ea4e2ae8b27,d61b915b69851aec8a8865f36cfd793e,2017-03-21 19:25:23,2017-03-21 19:25:22,0.00
6,011575986092c30523ecb71ff10cb473,ed0a6265b5b0a292cd537214f401f1fc,2018-04-18 21:58:08,2018-02-17 15:54:49,60.25
7,011b4adcd54683b480c4d841250a987f,93b35affb86435b71ca01a6dd1eab2b1,2018-02-15 11:40:57,2017-08-22 12:51:29,176.95
8,012452d40dafae4df401bced74cdb490,e49d0607832db7c3b1343764225d8df6,2018-05-14 12:12:45,2017-06-18 22:46:42,329.56
9,012a218df8995d3ec3bb221828360c86,207258d556f9b9784b54de837ae49286,2018-06-18 13:08:38,2018-05-07 10:28:17,42.11


In [ ]:

query = """
    SELECT
        product_category_name AS original_name,
        -- REPLACE() swaps all occurrences of one string with another.
        -- We will also capitalize the first letter of each word, which is harder in SQLite
        -- but REPLACE is the key first step.
        REPLACE(product_category_name, '_', ' ') AS cleaned_name
    FROM
        products
    WHERE
        product_category_name IS NOT NULL
        AND product_category_name LIKE '%_saude' -- Find categories ending in '_saude' (health)
    LIMIT 10;
"""

cleaned_names = pd.read_sql_query(query, conn)
print("Cleaning product category names:")
cleaned_names

Cleaning product category names:


,original_name,cleaned_name
0,beleza_saude,beleza saude
1,beleza_saude,beleza saude
2,beleza_saude,beleza saude
3,beleza_saude,beleza saude
4,beleza_saude,beleza saude
5,beleza_saude,beleza saude
6,beleza_saude,beleza saude
7,beleza_saude,beleza saude
8,beleza_saude,beleza saude
9,beleza_saude,beleza saude


In [ ]:
query = """
    SELECT
        -- STRFTIME creates a formatted string from a date/time column.
        -- '%Y-%m' gives us a 'YYYY-MM' format perfect for grouping.
        STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
        SUM(oi.price) AS total_monthly_revenue
    FROM
        orders AS o
    JOIN
        order_items AS oi ON o.order_id = oi.order_id
    WHERE
        STRFTIME('%Y', o.order_purchase_timestamp) = '2017'
    GROUP BY
        year_month
    ORDER BY
        year_month;
"""

monthly_revenue_2017 = pd.read_sql_query(query, conn)
print("Total Monthly Revenue for 2017:")
monthly_revenue_2017

Total Monthly Revenue for 2017:


,year_month,total_monthly_revenue
0,2017-01,120312.87
1,2017-02,247303.02
2,2017-03,374344.30
3,2017-04,359927.23
4,2017-05,506071.14
5,2017-06,433038.60
6,2017-07,498031.48
7,2017-08,573971.68
8,2017-09,624401.69
9,2017-10,664219.43


In [ ]:
query = """
    -- Step 1: Create a CTE to calculate the monthly revenue for each category.
    -- This joins all the tables and performs the initial aggregation.
    WITH monthly_category_revenue AS (
        SELECT
            STRFTIME('%Y-%m', o.order_purchase_timestamp) AS year_month,
            p.product_category_name,
            SUM(oi.price) AS monthly_revenue
        FROM
            order_items AS oi
        JOIN
            orders AS o ON oi.order_id = o.order_id
        JOIN
            products AS p ON oi.product_id = p.product_id
        WHERE
            STRFTIME('%Y', o.order_purchase_timestamp) = '2017'
            AND p.product_category_name IS NOT NULL
        GROUP BY
            year_month, p.product_category_name
    ),
    -- Step 2: Create a second CTE to rank the categories within each month based on revenue.
    -- This is where we use a window function on the result of our first step.
    ranked_monthly_revenue AS (
        SELECT
            year_month,
            product_category_name,
            monthly_revenue,
            RANK() OVER (
                PARTITION BY year_month        -- Look at each month independently
                ORDER BY monthly_revenue DESC  -- Order by the revenue we just calculated
            ) as category_rank
        FROM
            monthly_category_revenue
    )
    -- Step 3: Select only the #1 ranked category for each month from the final CTE.
    SELECT
        year_month,
        product_category_name AS top_category,
        ROUND(monthly_revenue, 2) AS monthly_revenue
    FROM
        ranked_monthly_revenue
    WHERE
        category_rank = 1
    ORDER BY
        year_month;
"""

monthly_winners = pd.read_sql_query(query, conn)
print("Top performing product category for each month in 2017:")
monthly_winners

Top performing product category for each month in 2017:


,year_month,top_category,monthly_revenue
0,2017-01,moveis_decoracao,13521.31
1,2017-02,beleza_saude,22838.79
2,2017-03,informatica_acessorios,28624.60
3,2017-04,esporte_lazer,24864.33
4,2017-05,beleza_saude,46786.02
5,2017-06,cool_stuff,38244.50
6,2017-07,cama_mesa_banho,63888.75
7,2017-08,cama_mesa_banho,57137.23
8,2017-09,pcs,52878.88
9,2017-10,relogios_presentes,65959.53
